In [302]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
sys.path.insert(0, '../')
sys.path.insert(0,"../../scripts/")

import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 999)

# import textract

from tabula import read_pdf
import camelot

%reload_ext autoreload
%autoreload 2



# from scripts import manipulate

In [303]:
dd = read_pdf('../../files/ldo/ldo_2021.pdf', pages='121-268' ,multiple_tables=True, encoding='utf-8', lattice=False)

In [326]:
df = pd.DataFrame()

for i in range(len(dd)):
    if len(dd[i].columns)>2:
        print(i)
        dd[i] = dd[i][dd[i].columns[:2]]
    dd[i].columns = [0,1]
    df = pd.concat([df,dd[i]], axis=0)
    
df = df.reset_index(drop=True)

mask = (df[1].fillna('').str.contains(r'\b\d', regex=True)) & (~df[1].fillna('').str.contains(r'META', regex=True))
df[1] = np.where(mask,df[1],np.nan)
mask = (df[1].fillna('').str.contains(r'\b\d\d\d\d', regex=True))
df[0] = np.where(mask,df[0] +'\n' +df[1] ,df[0])
df[1] = np.where(mask,np.nan,df[1])

In [327]:
# def re_filter(value):  
#     return re.findall(_filter, value)
def generate_col(df,_filter):
    mask = df[0].fillna('').str.contains(_filter,regex=True, case=True)
    return df[mask][0]

def fill_col(df, _filter):
    mask = df['root'].fillna('').str.contains(_filter,regex=True, case=True)
    return np.where(mask, df[0],np.nan)

In [331]:
df["programa"] = generate_col(df, r"\bPrograma:\n")
df["orgao"] = generate_col(df, r"\bÓrgão:\n")
df["indicador_programa"] = generate_col(df, r"\bINDICADORES DE RESULTADO DE PROGRAMA")
df["produto"] = generate_col(df, r"\bPRODUTO: ")
df["indicador_produto"] = generate_col(df, r"\bINDICADOR DE PRODUTO")
df["acao"] = generate_col(df, r"\bAÇÃO\b")

df["root"] = (
    df["programa"].fillna("")
    + df["orgao"].fillna("")
    + df["indicador_programa"].fillna("")
    + df["produto"].fillna("")
    + df["indicador_produto"].fillna("")
    + df["acao"].fillna("")
)
df["root"] = np.where(df["root"] == "", np.nan, df["root"])
df["root"] = df["root"].fillna(method="ffill")



df["programa"] = fill_col(df, r"\bPrograma:\n")
df["orgao"] = fill_col(df, r"\bÓrgão:\n")
df["indicador_programa"] = fill_col(df, r"\bINDICADORES DE RESULTADO DE PROGRAMA")
df["produto"] = fill_col(df, r"\bPRODUTO: ")
df["indicador_produto"] = fill_col(df, r"\bINDICADOR DE PRODUTO")
df["acao"] = fill_col(df, r"\bAÇÃO\b")
df["acao"] = df[
    df["acao"].fillna("").str.contains(r"\b\d\d\d\d -", regex=True, case=True)
]["acao"]




finalidade_progama_mask = (
    df["programa"].fillna("").str.contains(r"\bPrograma:\n", regex=True, case=True)
)
df["finialidade_programa"] = df[~finalidade_progama_mask]["programa"]
df["programa"] = np.where(finalidade_progama_mask, df["programa"], np.nan)

finalidade_produto_mask = (
    df["produto"].fillna("").str.contains(r"\bPRODUTO: ", regex=True, case=True)
)
df["finialidade_produto"] = df[~finalidade_produto_mask]["produto"]
df["produto"] = np.where(finalidade_produto_mask, df["produto"], np.nan)


for col in df.columns[2:]:
    if col != "acao":
        df[col] = df[col].fillna(method="ffill")
    else:
        df[col] = df[col].fillna(method="bfill")


value_programa_mask = (
    df["root"]
    .fillna("")
    .str.contains(r"\bINDICADORES DE RESULTADO DE PROGRAMA", regex=True, case=True)
)
df["value_indicador_programa"] = np.where(value_programa_mask, df[1], np.nan)
df["value_indicador_programa"] = (
    df["value_indicador_programa"].str.replace(".", "").str.replace(",", ".")
)
df["value_indicador_programa"] = pd.to_numeric(df["value_indicador_programa"], errors="coerce")


In [332]:


value_produto_mask = (
    df["root"].fillna("").str.contains(r"\bINDICADOR DE PRODUTO", regex=True, case=True)
)
df["value_indicador_produto"] = np.where(value_produto_mask, df[1], np.nan)
df["value_indicador_produto"] = (
    df["value_indicador_produto"].str.replace(".", "").str.replace(",", ".")
)
df["value_indicador_produto"] = pd.to_numeric(df["value_indicador_produto"], errors="coerce",downcast='float')


df_programa = df[df["value_indicador_programa"].notnull()]
cols = [
    "programa",
    "orgao",
    "indicador_programa",
    "finialidade_programa",
    "value_indicador_programa",
]
df_programa = df_programa[cols]
df_programa['programa'] = df_programa['programa'].str.replace('Programa:\n','')
df_programa['orgao'] = df_programa['orgao'].str.replace('Órgão:\n','')
for col in df_programa.columns[:-1]:
    df_programa[col] = df_programa[col].str.upper().str.replace('\r','')

    
    
cols = [
    "programa",
    "orgao",
    "finialidade_programa",
    "produto",
    "finialidade_produto",
    "indicador_produto",
    "acao",
    "value_indicador_produto",
]
df_produto = df[df["value_indicador_produto"].notnull()]
df_produto = df_produto[cols]

df_produto['programa'] = df_produto['programa'].str.replace('Programa:\n','')
df_produto['orgao'] = df_produto['orgao'].str.replace('Órgão:\n','')
for col in df_produto.columns[:-1]:
    df_produto[col] = df_produto[col].str.upper().str.replace('\r','')

In [333]:
df_programa.to_csv('../../data/ldo/ldo_2021_programa.csv', index=False)
df_produto.to_csv('../../data/ldo/ldo_2021_produto.csv', index=False)

In [336]:
df_produto['value_indicador_produto'].apply(lambda x : 2*x)

17      6.100000e+02
24      2.000000e+00
31      1.200000e+03
38      5.760000e+03
45      6.000000e+01
52      1.600000e+01
65      2.400000e+02
72      4.928000e+04
92      6.578000e+03
99      8.000000e+01
106     4.000000e+00
127     9.326498e+06
134     9.883562e+06
135     1.628000e+02
142     1.384800e+02
143     3.203816e+06
150     2.694000e+05
157     1.300000e+03
164     4.258500e+05
177     3.400000e+01
184     1.120000e+05
211     1.059200e+04
218     1.200000e+01
240     1.743200e+02
241     1.940000e+02
248     7.286000e+03
249     1.590494e+06
256     6.646702e+06
266     8.920000e+05
267     2.000000e+02
274     6.646702e+06
275     2.000000e+02
289     4.075510e+06
296     1.172300e+05
297     7.678000e+01
304     6.160200e+05
305     1.470000e+02
312     1.920000e+02
313     1.560000e+02
320     1.700000e+02
327     1.142800e+02
334     2.000000e+00
356     4.838800e+06
363     2.000000e+01
370     2.000000e+03
371     2.000000e+02
378     1.000000e+02
385     6.000

In [301]:
df_programa[df_programa['programa']=='1218 - FOMENTO CULTURAL']

,programa,orgao,indicador_programa,finialidade_programa,value_indicador_programa
1398,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,NÚMERO DE EDITAIS LANÇADOS PELO PROGRAMA (UNID...,FINALÍSTICO,33.0
1399,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,NÚMERO DE MUNICÍPIOS CONTEMPLADOS NO PROGRAMA ...,FINALÍSTICO,180.0
1400,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,VALOR APORTADO NO PROGRAMA DE FOMENTO CULTURAL...,FINALÍSTICO,112360000.0
